In [2]:
import torch
from pathlib import Path
from tqdm import tqdm
from PIL import Image
from transformers import BlipForQuestionAnswering, BlipProcessor
from torch.utils.data import DataLoader, Dataset

d:\project\student_resource 3\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MAX_LENGTH=20
BATCH_SIZE=8
NUM_EPOCHS=20
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
BASE_DIR=Path('D:\\project\\student_resource 3')
%cd {BASE_DIR}

cuda
D:\project\student_resource 3


d:\project\student_resource 3\env\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
# model_name = "Salesforce/blip-vqa-base"
# model = BlipForQuestionAnswering.from_pretrained(model_name)
# processor = BlipProcessor.from_pretrained(model_name)

In [4]:
save_path = f"{BASE_DIR}/models/finetuned_blip_model"
model = BlipForQuestionAnswering.from_pretrained(save_path)
processor = BlipProcessor.from_pretrained(save_path)

In [5]:
class VQADataset(Dataset):
    def __init__(self, dataframe, processor, max_length=MAX_LENGTH):
        self.dataframe = dataframe
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        question = row["question"]
        answer = row["answer"]

        inputs = self.processor(
            text=question,
            images=image,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )
        inputs["labels"] = self.processor.tokenizer(
            answer,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        ).input_ids

        for key in inputs:
            inputs[key] = inputs[key].squeeze(0)

        return inputs

In [6]:
import pandas as pd

df = pd.read_csv(f"{BASE_DIR}/data/train_vqa.csv")
train_dataset = VQADataset(df, processor)

In [7]:
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
model = model.to(device)

In [9]:
import os

ckpt_dir = "./checkpoints"
os.makedirs(ckpt_dir, exist_ok=True)

In [10]:
for epoch in range(21, 21 + NUM_EPOCHS):
    model.train()
    total_loss = 0
    for batch in tqdm(train_dataloader):
        input_ids = batch["input_ids"].to(device)
        pixel_values = batch["pixel_values"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(
            input_ids=input_ids,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            labels=labels,
        )
        loss = outputs.loss
        total_loss += loss.item()

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    avg_loss = total_loss / len(train_dataloader)
    print(f"Epoch {epoch + 1}/{NUM_EPOCHS}, Loss: {avg_loss}")
    checkpoint_path = os.path.join(ckpt_dir, f"epoch_{epoch+1}.pth")
    torch.save(
        {
            "epoch": epoch + 1,
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss": avg_loss,
        },
        checkpoint_path,
    )
    print(f"Checkpoint saved at {checkpoint_path}")
    print()

100%|██████████| 2000/2000 [29:56<00:00,  1.11it/s]


Epoch 22/20, Loss: 0.3875201988518238
Checkpoint saved at ./checkpoints\epoch_22.pth



100%|██████████| 2000/2000 [25:56<00:00,  1.28it/s]


Epoch 23/20, Loss: 0.382313860565424
Checkpoint saved at ./checkpoints\epoch_23.pth



100%|██████████| 2000/2000 [28:45<00:00,  1.16it/s]


...

Epoch 39/20, Loss: 1.6156916198134423
Checkpoint saved at ./checkpoints\epoch_39.pth



100%|██████████| 2000/2000 [25:28<00:00,  1.31it/s]


Epoch 40/20, Loss: 1.6138958581089973
Checkpoint saved at ./checkpoints\epoch_40.pth



100%|██████████| 2000/2000 [25:29<00:00,  1.31it/s]


Epoch 41/20, Loss: 1.6139339218735695
Checkpoint saved at ./checkpoints\epoch_41.pth



In [11]:
save_path = f"{BASE_DIR}/models/finetuned_blip_model"
model.save_pretrained(save_path)
processor.save_pretrained(save_path)

[]

# Testing

In [5]:
class VQATestDataset(Dataset):
    def __init__(self, dataframe, processor, max_length=MAX_LENGTH):
        self.dataframe = dataframe
        self.processor = processor
        self.max_length = max_length

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        image = Image.open(row["image_path"]).convert("RGB")
        question = row["question"]
        answer = row["answer"]  # Ground-truth answer for evaluation

        inputs = self.processor(
            text=question,
            images=image,
            max_length=self.max_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt",
        )

        for key in inputs:
            inputs[key] = inputs[key].squeeze(0)

        return {
            "pixel_values": inputs["pixel_values"],
            "input_ids": inputs["input_ids"],
            "attention_mask": inputs["attention_mask"],
            "question": question,
            "answer": answer,  # For evaluation purposes
        }

In [ ]:
import pandas as pd

test_df = pd.read_csv(f"{BASE_DIR}/data/test_vqa.csv")

In [8]:
test_dataset = VQATestDataset(test_df, processor)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [9]:
from evaluate import load

metric = load("sacrebleu")

In [ ]:
model = BlipForQuestionAnswering.from_pretrained(save_path)
processor = BlipProcessor.from_pretrained(save_path)

In [20]:
model = model.to(device)

In [ ]:
model.load_state_dict(
    torch.load(f"{BASE_DIR}/checkpoints/epoch_31.pth")["model_state_dict"]
)

C:\Users\CSE IIT BHILAI\AppData\Local\Temp\ipykernel_1628\1847902965.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{BASE_DIR}/checkp

<All keys matched successfully>

In [ ]:
model = model.to(device)

In [14]:
model.eval()

predictions = []
references = []

with torch.no_grad():
    for batch in tqdm(test_dataloader):
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)

        # Generate predictions
        outputs = model.generate(
            input_ids=input_ids,
            pixel_values=pixel_values,
            attention_mask=attention_mask,
            max_length=MAX_LENGTH,  # Ensure this matches your model configuration
            num_beams=5,  # Beam search for better quality answers
        )

        # Decode predictions and ground-truth answers
        predicted_answer = processor.tokenizer.batch_decode(
            outputs, skip_special_tokens=True
        )
        ground_truth_answer = batch["answer"]

        predictions.append(predicted_answer)
        references.append(ground_truth_answer)  # List of lists for BLEU metric

        metric.add_batch(predictions=predicted_answer, references=ground_truth_answer)

        # print(f"Q: {batch['question'][0]}")
        # print(f"Predicted: {predicted_answer}")
        # print(f"Ground Truth: {ground_truth_answer}\n")

# Evaluate the BLEU score
results = metric.compute()
print(f"BLEU Score: {results['score']}")

  0%|          | 0/500 [00:00<?, ?it/s]

100%|██████████| 500/500 [06:41<00:00,  1.25it/s]


BLEU Score: 0.10159281170372051


In [15]:
import numpy as np
from itertools import chain

In [17]:
results_df = pd.DataFrame()
results_df["labels"] = list(chain.from_iterable(references))
results_df["predictions"] = list(chain.from_iterable(predictions))
results_df.head()

,labels,predictions
0,6.5 kilogram,200. 0 millilitre
1,96.0 watt,10. 0 centimetre
2,230.0 millimetre,5. 0 volt
3,10.0 centimetre,12. 0 centimetre
4,60.0 watt,"[ 220. 0, 240. 0 ] volt"


In [18]:
results_df.to_csv(f"{BASE_DIR}/results/finetuned_blip.csv", index=False)